In [ ]:
import numpy as np
import itertools
import astropy.cosmology as cosmology
import scipy.optimize as spo
import scipy.stats as sps
from scipy.stats import norm
import scipy.linalg as la
import scipy as sp
import emcee
#import corner
from datetime import datetime
import hickle
import bisect
import daft
import cProfile
from io import StringIO
import pstats
import sys
import pylab as pl
from sklearn.metrics import confusion_matrix
epsilon = sys.float_info.min
log_epsilon = sys.float_info.min_exp

import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
from matplotlib import rc
rc("font", family="serif", size=12)
rc("text", usetex=True)
colors = 'rbgcymk'
import h5py
#h5py.run_tests()

In [ ]:
def safe_log(arr, threshold=epsilon):
    shape = np.shape(arr)
    flat = arr.flatten()
    logged = np.log(np.array([max(a, threshold) for a in flat])).reshape(shape)
    return logged

def reg_vals(arr, threshold=log_epsilon):
    arr[arr < threshold] = threshold
    return arr

In [ ]:
hickle.load_dataset
with open('data/data.hkl', 'r+') as in_file:
    sim_info = hickle.load(in_file)

with open('data/truth.hkl', 'r+') as in_file:
    truth_info = hickle.load(in_file)
    
    
print('all keys in truth: ')
print(truth_info.keys())
    
true_data = truth_info['data']
#print(true_data)

true_zs = [true_param['z'] for true_param in true_data]

print('n_zs truth: ', len(true_zs))
true_mus = [true_param['mu'] for true_param in true_data]

true_types = [true_param['t'] for true_param in true_data]
true_id = truth_info['id']

#true_types = true_data['t']
#true_z = true_data['z']
#true_mu=true_data['mu']

print('all keys in data: ')
print(sim_info.keys())

types = sim_info['types']
n_types = len(types)
print('n_types: ', n_types)

z_bins = sim_info['z_bins']
z_difs = z_bins[1:] - z_bins[:-1]
z_mids = (z_bins[1:] + z_bins[:-1]) / 2.
n_zs = len(z_difs)
sn_id = sim_info['id']


print('n_zs: ', n_zs)

mu_bins = sim_info['mu_bins']

ln_int_post = sim_info.get('interim ln posteriors')
num_sne = np.shape(ln_int_post)[0]

print(np.shape(ln_int_post), 'shape post')
print('num_sne: ', num_sne)


#pl.figure(100)
#for s in range(num_sne):
    #print(true_id[s], true_zs[s], true_mus[s], sn_id[s], z_mids[s], mu_mids[s])
#    pl.plot(true_zs[s], z_bins[s] , marker='.', linestyle='None')
    
    
#pl.xlabel('true z')
#pl.ylabel('data z')
mu_difs = mu_bins[1:] - mu_bins[:-1]
#print(z_bins)
#print(z_difs)
mu_mids = (mu_bins[1:] + mu_bins[:-1]) / 2.
n_mus = len(mu_difs)
print('n_mus: ', n_mus)



fig = plt.figure(figsize=(n_types*6, 5))
p = 0
for s in [1,7,0]:
#range(5):
    fig = plt.figure(figsize=(n_types*6, 5))
    print(true_id[s], true_zs[s], true_mus[s], sn_id[s], z_mids[s], mu_mids[s])
    p = 0
    for t in range(3):
        p += 1
        
        #print(sn_id[s], true_id[s])
        plt.subplot(1, n_types, p)
        plt.pcolormesh(z_bins, mu_bins, ln_int_post[s,t,:,:].T, cmap='viridis')
        plt.scatter(true_zs[s], true_mus[s], marker='.', color='r', linestyle='None')
        plt.title(r'data: $%s$ Truth: $%s$'%(sn_id[s], true_id[s]))

In [ ]:
print(true_types)

## Integrating over both mu and z to get P(type)

In [ ]:
type_tmp=np.zeros((num_sne,3))

# Making sure to convert to posterior (not log post) and then do simple integral. This should be improved later
# scipy.integrate.simps(y, x=None, dx=1, axis=-1, even='avg')

#for zi in range(1,n_zs):
#    for mui in range(1,n_mus):
        
type_tmp[:,:] =np.dot(np.dot(np.exp(ln_int_post[:,:,:,:]),mu_difs), z_difs)
        
        #*(z_bins[zi]-z_bins[zi-1])*(mu_bins[mui]-mu_bins[mui-1])


# Ensure that the probabilities are normalised
norm = np.sum(type_tmp, axis=1)
type_tmp[:,0]=type_tmp[:,0]/norm
type_tmp[:,1]=type_tmp[:,1]/norm
type_tmp[:,2]=type_tmp[:,2]/norm


## Now we assign the true type by the highest probability.

In [ ]:
type_converter = {'Ia': 0, 'Ibc': 1, 'II': 2}
true_types_indices = [type_converter[m] for m in true_types]
true_type = np.argmax(type_tmp, axis=1)

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# conf_mat = skl.metrics.confusion_matrix(true_types_indices, true_type)
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(true_types_indices, true_type)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=types,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=types, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
# true_types_indices = [type_converter[m] for m in true_types]
# true_type = np.argmax(type_tmp, axis=1)# Planck 2015 results XIV. Dark energy and modified gravity - Figure 3

# plt.plot(true_types_indices, true_type)
# conf_mat = skl.metrics.confusion_matrix(true_types_indices, true_type)
true_H0 = 67.9
true_Ode0 = 0.693
true_Om0 = 1. - true_Ode0
true_w0 = -1.09
true_wa = -0.20
true_hyperparams = np.array([true_w0, true_wa])
n_hyperparams = len(true_hyperparams)
#true_cosmo = cosmology.FlatLambdaCDM(H0=true_H0, Om0=true_Om0)
true_cosmo = cosmology.w0waCDM(true_H0, true_Om0, true_Ode0, w0=true_w0, wa=true_wa)

#for n in range(num_sne):
#    true_params[n]['mu'] = true_cosmo.distmod(true_params[n]['z']).value

In [ ]:
a = np.array([[10, 7, 4], [3, 2, 1]])
print(a)
print(np.percentile(a, 50, axis=0))
print(np.percentile(a, 50, axis=1))

Let's make a Hubble plot assuming every SN is Ia:

In [ ]:
mumed = mumedia=np.zeros(num_sne)
mulow = mulowia=np.zeros(num_sne)
zhigh = zhighia=np.zeros(num_sne)
zlow = zlowia=np.zeros(num_sne)
muhigh = muhighia=np.zeros(num_sne)
zmed = zmedia=np.zeros(num_sne)
from scipy.interpolate import interp1d

pmin, pmax = log_epsilon, np.log(1./(min(z_difs) * min(mu_difs)))

for sn in range(num_sne):
    
    #pl.imshow(ln_int_post[sn,true_type[sn],:,:])
    
    #plt.colorbar()
    #pl.show()
    #pl.figure()
    #plt.pcolormesh(z_bins, mu_bins, ln_int_post[sn,true_type[sn],:,:].T, cmap='viridis', vmin = pmin, vmax = pmax)

    # True type
    #print(true_type[sn])
    hm = ln_int_post[sn,true_type[sn],:,:].T
    #print(np.shape(hm), np.shape(z_difs))
    mu_prob =np.dot(np.exp(hm),z_difs)
    muspline = interp1d(mu_prob, mu_bins[:-1])
    indsmu=np.argsort(mu_prob)
    
    #print(np.shape(mu_prob), np.shape(mu_bins))
    
    #mu_bins[np.argmax(mu_prob)]
    
    #indsmu=np.argsort(mu_prob)

    #print(np.percentile(mu_prob,0.16,interpolation='linear'), '16')
    #print(muspline(np.percentile(mu_prob,0.16,interpolation='linear')))
    
    #print(np.percentile(mu_prob,0.5,interpolation='linear'), '50')
    #print(muspline(np.percentile(mu_prob,0.5,interpolation='linear')))
    
    
    #print(np.percentile(mu_prob,0.84,interpolation='linear'), '84')
    #print(muspline(np.percentile(mu_prob,0.84,interpolation='linear')))
    
    mumed[sn]= mu_bins[np.argmax(mu_prob)]
    #muspline(np.percentile(mu_prob[indsmu],0.5,interpolation='linear'))
    mulow[sn] = muspline(np.percentile(mu_prob[indsmu],0.95,interpolation='linear'))
    muhigh[sn] = muspline(np.percentile(mu_prob[indsmu],0.05,interpolation='linear'))
    #print(mulow[sn],mumed[sn],muhigh[sn])
            
       
    zm = np.exp(ln_int_post[sn,true_type[sn],:,:])
    z_prob = np.dot(zm,mu_difs[:])
    #print(z_prob)
    indsz=np.argsort(z_prob)

    zspline = interp1d(z_prob, z_bins[:-1])
    
    zmed[sn]= z_bins[np.argmax(z_prob)]
    zlow[sn] = zspline(np.percentile(z_prob[indsz],0.95,interpolation='linear'))
    zhigh[sn] = zspline(np.percentile(z_prob[indsz],0.05,interpolation='linear'))
    
    # Ia type
    hmia = ln_int_post[sn,0,:,:].T
    mu_probia =np.dot(np.exp(hmia),z_difs)
    mumedia[sn]= mu_bins[np.argmax(mu_probia)]
    musplineia = interp1d(mu_probia, mu_bins[:-1])
    indsmuia=np.argsort(mu_probia)

    
    #muspline(np.percentile(mu_prob[indsmu],0.5,interpolation='linear'))
    mulowia[sn] = musplineia(np.percentile(mu_probia[indsmuia],0.95,interpolation='linear'))
    muhighia[sn] = musplineia(np.percentile(mu_probia[indsmuia],0.05,interpolation='linear'))

    #print(true_type[sn], mumedia[sn], mumed[sn]) 
    zmia = np.exp(ln_int_post[sn,0,:,:])
    z_prob = np.dot(zmia,mu_difs[:])
    #print(z_prob)
    indszia=np.argsort(z_prob)

    zspline = interp1d(z_prob, z_bins[:-1])
    
    zmedia[sn]= z_bins[np.argmax(z_prob)]
    zlowia[sn] = zspline(np.percentile(z_prob[indszia],0.95,interpolation='linear'))
    zhighia[sn] = zspline(np.percentile(z_prob[indszia],0.05,interpolation='linear'))

    
muerr=np.max([mumed[sn]-mulow[sn], muhigh[sn]-mumed[sn]])
zerr=np.max([zmed[sn]-zlow[sn], zhigh[sn]-zmed[sn]])
muerria=np.max([mumedia[sn]-mulowia[sn], muhighia[sn]-mumedia[sn]])
zerria=np.max([zmedia[sn]-zlowia[sn], zhighia[sn]-zmedia[sn]])
    #print(zerr,muerr)

indsz = np.argsort(zmed)
pl.figure(10,figsize=(8,6))
plt.errorbar(zmed[indsmu], mumed[indsmu],yerr=muerr ,xerr=zerr, marker='o', linestyle='None', color='teal')
#print(true_zs, indsmu)

for ic, ind in enumerate(indsmu):
    
    plt.scatter(true_zs[ind], true_mus[ind], marker='.', color='r', linestyle='None')
#plt.errorbar(zmedia[indsmuia], mumedia[indsmuia],yerr=muerria ,xerr=zerria, marker='o', linestyle='None', color='orange')

#print(zmed,mumed)
#plt.plot(zmed[indsz],mumed[indsz])
pl.title('assuming every SN is Ia')
pl.xlabel('z',fontsize=16)
pl.ylabel(r'$\mu(z)$', fontsize=16)
pl.savefig('fakeHubble_all.png')
#print(len(zmed))
#print(mumed,mumedia)
#pl.plot(zmed[mumed>0], mumed[mumed>0]-mumedia[mumed>0], '.')
#print(np.max(zmed))
    

#for zi in range(1, n_zs):
#    muvec[zi] +=np.exp(ln_int_post[:,true_type[zi],zi,:])*(z_bins[zi]-z_bins[zi-1])

#print muvec


In [ ]:
mumed = mumedia=np.zeros(num_sne)
mulow = mulowia=np.zeros(num_sne)
zhigh = zhighia=np.zeros(num_sne)
zlow = zlowia=np.zeros(num_sne)
muhigh = muhighia=np.zeros(num_sne)
zmed = zmedia=np.zeros(num_sne)
from scipy.interpolate import interp1d

pmin, pmax = log_epsilon, np.log(1./(min(z_difs) * min(mu_difs)))

for sn in np.where(true_type==0)[0]:#range(num_sne):
    
    #pl.imshow(ln_int_post[sn,true_type[sn],:,:])
    
    #plt.colorbar()
    #pl.show()
    #pl.figure()
    #plt.pcolormesh(z_bins, mu_bins, ln_int_post[sn,true_type[sn],:,:].T, cmap='viridis', vmin = pmin, vmax = pmax)

    # True type
    #print(true_type[sn])
    hm = ln_int_post[sn,true_type[sn],:,:].T
    #print(np.shape(hm), np.shape(z_difs))
    mu_prob =np.dot(np.exp(hm),z_difs)
    muspline = interp1d(mu_prob, mu_bins[:-1])
    indsmu=np.argsort(mu_prob)
    
    #print(np.shape(mu_prob), np.shape(mu_bins))
    
    #mu_bins[np.argmax(mu_prob)]
    
    #indsmu=np.argsort(mu_prob)

    #print(np.percentile(mu_prob,0.16,interpolation='linear'), '16')
    #print(muspline(np.percentile(mu_prob,0.16,interpolation='linear')))
    
    #print(np.percentile(mu_prob,0.5,interpolation='linear'), '50')
    #print(muspline(np.percentile(mu_prob,0.5,interpolation='linear')))
    
    
    #print(np.percentile(mu_prob,0.84,interpolation='linear'), '84')
    #print(muspline(np.percentile(mu_prob,0.84,interpolation='linear')))
    
    mumed[sn]= mu_bins[np.argmax(mu_prob)]
    #muspline(np.percentile(mu_prob[indsmu],0.5,interpolation='linear'))
    mulow[sn] = muspline(np.percentile(mu_prob[indsmu],0.95,interpolation='linear'))
    muhigh[sn] = muspline(np.percentile(mu_prob[indsmu],0.05,interpolation='linear'))
    #print(mulow[sn],mumed[sn],muhigh[sn])
            
       
    zm = np.exp(ln_int_post[sn,true_type[sn],:,:])
    z_prob = np.dot(zm,mu_difs[:])
    #print(z_prob)
    indsz=np.argsort(z_prob)

    zspline = interp1d(z_prob, z_bins[:-1])
    
    zmed[sn]= z_bins[np.argmax(z_prob)]
    zlow[sn] = zspline(np.percentile(z_prob[indsz],0.95,interpolation='linear'))
    zhigh[sn] = zspline(np.percentile(z_prob[indsz],0.05,interpolation='linear'))
    
    # Ia type
    hmia = ln_int_post[sn,0,:,:].T
    mu_probia =np.dot(np.exp(hmia),z_difs)
    mumedia[sn]= mu_bins[np.argmax(mu_probia)]
    musplineia = interp1d(mu_probia, mu_bins[:-1])
    indsmuia=np.argsort(mu_probia)

    
    #muspline(np.percentile(mu_prob[indsmu],0.5,interpolation='linear'))
    mulowia[sn] = musplineia(np.percentile(mu_probia[indsmuia],0.95,interpolation='linear'))
    muhighia[sn] = musplineia(np.percentile(mu_probia[indsmuia],0.05,interpolation='linear'))

    #print(true_type[sn], mumedia[sn], mumed[sn]) 
    zmia = np.exp(ln_int_post[sn,0,:,:])
    z_prob = np.dot(zmia,mu_difs[:])
    #print(z_prob)
    indszia=np.argsort(z_prob)

    zspline = interp1d(z_prob, z_bins[:-1])
    
    zmedia[sn]= z_bins[np.argmax(z_prob)]
    zlowia[sn] = zspline(np.percentile(z_prob[indszia],0.95,interpolation='linear'))
    zhighia[sn] = zspline(np.percentile(z_prob[indszia],0.05,interpolation='linear'))

    
muerr=np.max([mumed[sn]-mulow[sn], muhigh[sn]-mumed[sn]])
zerr=np.max([zmed[sn]-zlow[sn], zhigh[sn]-zmed[sn]])
muerria=np.max([mumedia[sn]-mulowia[sn], muhighia[sn]-mumedia[sn]])
zerria=np.max([zmedia[sn]-zlowia[sn], zhighia[sn]-zmedia[sn]])
    #print(zerr,muerr)

indsz = np.argsort(zmed)
pl.figure(10,figsize=(8,6))
plt.errorbar(zmed[indsmu], mumed[indsmu],yerr=muerr ,xerr=zerr, marker='o', linestyle='None', color='teal')
#print(true_zs, indsmu)

for ic, ind in enumerate(indsmu):
    
    plt.scatter(true_zs[ind], true_mus[ind], marker='.', color='r', linestyle='None')
#plt.errorbar(zmedia[indsmuia], mumedia[indsmuia],yerr=muerria ,xerr=zerria, marker='o', linestyle='None', color='orange')
#print(zmed,mumed)
#plt.plot(zmed[indsz],mumed[indsz])
pl.ylim(40., 45.)
plt.xlim(0.15, 0.65)
plt.title('only SN that are most likely to be Ia')
pl.xlabel('z',fontsize=16)
pl.ylabel(r'$\mu(z)$', fontsize=16)
pl.savefig('fakeHubble_assigned_Ia.png')
#print(len(zmed))
#print(mumed,mumedia)
#pl.plot(zmed[mumed>0], mumed[mumed>0]-mumedia[mumed>0], '.')
#print(np.max(zmed))
    

#for zi in range(1, n_zs):
#    muvec[zi] +=np.exp(ln_int_post[:,true_type[zi],zi,:])*(z_bins[zi]-z_bins[zi-1])

#print muvec